In [ ]:
import json
import glob
from os import listdir
from os.path import isfile, join
import os.path as osp
import copy
import os
import pickle
from datetime import datetime
import random
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.filtering.log.attributes import attributes_filter
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv
from torch_geometric.nn import GATv2Conv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
interest_path = r'D:\Final master thesis evaluation\datasets'
log_pattern = r'D:\Final master thesis evaluation\datasets\*.json'
logs = glob.glob(log_pattern)
allfiles = [f for f in listdir(interest_path) if isfile(join(interest_path, f))]
allfilescopy = copy.deepcopy(allfiles)
allfiles_str = sorted(list({x.replace('.json', '.xes') for x in allfiles}))
label_str = sorted(list({x.replace('.json', '.csv') for x in allfilescopy}))
def getListofallkeys(dict):      
    return list(dict.keys())
log_counter = 0
for log in logs:
    # Opening JSON file
    f = open(log,)
    # returns JSON object as a dictionary
    data = json.load(f) 
    # Closing file
    f.close()
    #print(type(data))
    #print(getListofallkeys(data))
    all_case = data.get('cases', None)
    num_cases = len(all_case)
    #print(type(all_case), num_cases, all_case[0], type(all_case[0]))
    temp_list = all_case[0].get('events', None)
    att_dic = temp_list[0].get('attributes', None)
    list_of_attributes = getListofallkeys(att_dic)
    event_list_to_df = []
    case_list_to_df = []
    normal_cases = 0
    anomalous_cases = 0
    timestamp = True
    for i in range (num_cases):
        label_hint = all_case[i].get('attributes', None)
        label = label_hint.get('label', None)
        if label != 'normal':
            anomalous_cases += 1
        else:
            normal_cases += 1
        events_hint = all_case[i].get('events', None)
        num_events = len(events_hint)
        case_id = all_case[i].get('id', None)
        for j in range (num_events):
            event_activity = events_hint[j].get('name', None)
            event_timestamp = events_hint[j].get('timestamp', None)
            event_timestamp_end = events_hint[j].get('timestamp_end', None)
            event_user_hint = events_hint[j].get('attributes', None)
            if len(list_of_attributes) == 4:
                event_user = event_user_hint.get('user', None)
                event_day = event_user_hint.get('day', None)
                event_country = event_user_hint.get('country', None)
                event_company = event_user_hint.get('company', None)
                event_dic = {'case_id' : case_id, 'concept:name' : event_activity , 'timestamp' : event_timestamp,
                             'timestamp_end': event_timestamp_end, 'user' : event_user, 'day' : event_day,
                            'country' : event_country, 'company': event_company}
            elif len(list_of_attributes) == 3:
                event_user = event_user_hint.get('user', None)
                event_day = event_user_hint.get('day', None)
                event_country = event_user_hint.get('country', None)
                event_dic = {'case_id' : case_id, 'concept:name' : event_activity , 'timestamp' : event_timestamp,
                             'timestamp_end': event_timestamp_end, 'user' : event_user, 'day' : event_day,
                            'country' : event_country}
            elif len(list_of_attributes) == 2:
                event_user = event_user_hint.get('user', None)
                event_day = event_user_hint.get('day', None)
                event_dic = {'case_id' : case_id, 'concept:name' : event_activity , 'timestamp' : event_timestamp,
                             'timestamp_end': event_timestamp_end, 'user' : event_user, 'day' : event_day}
            else:
                event_user = event_user_hint.get('user', None)
                event_dic = {'case_id' : case_id, 'concept:name' : event_activity , 'timestamp' : event_timestamp,
                             'timestamp_end': event_timestamp_end, 'user' : event_user}                
            event_list_to_df.append(event_dic)   
        label_dic = {'case_id' : case_id, 'label' : label}
        case_list_to_df.append(label_dic)
    event_df = pd.DataFrame.from_dict(event_list_to_df)
    label_df = pd.DataFrame.from_dict(case_list_to_df)
    if event_df['timestamp'].isnull().all():
        timestamp = False
    event_df.dropna(axis=1, how='all', inplace=True)
    print(normal_cases, anomalous_cases, timestamp)
    parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'case_id'}
    log = log_converter.apply(event_df, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)
    filename = allfiles_str[log_counter]
    full_filename = os.path.join(interest_path, filename)
    xes_exporter.apply(log, full_filename)
    filename = label_str[log_counter]
    full_filename = os.path.join(interest_path, filename)
    label_df.to_csv(full_filename)
    log_counter += 1